# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [368]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import pdist, squareform

In [369]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [370]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [378]:
df_q= data.groupby(['CustomerID', 'ProductName']).agg({'Quantity':'sum'}).reset_index()
df_q['Quantity']= df_q['Quantity'].astype(np.float64)
#df_q['CustomerID']= df_q['CustomerID'].astype('str')
df_q.head()

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1.0
1,33,Assorted Desserts,1.0
2,33,Bandage - Flexible Neon,1.0
3,33,"Bar Mix - Pina Colada, 355 Ml",1.0
4,33,"Beans - Kidney, Canned",1.0


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [379]:
df_pivot= pd.pivot_table(df_q, index="ProductName", columns='CustomerID', values="Quantity").fillna(0)
df_pivot.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [380]:
cs_matrix= squareform(pdist(df_pivot.T, 'euclidean'))
cs_matrix

array([[  0.        ,  11.91637529,  10.48808848, ..., 228.62851966,
        239.        , 229.77380181],
       [ 11.91637529,   0.        ,  11.74734012, ..., 228.01096465,
        239.03765394, 229.70415756],
       [ 10.48808848,  11.74734012,   0.        , ..., 228.08112592,
        238.26665734, 229.77380181],
       ...,
       [228.62851966, 228.01096465, 228.08112592, ...,   0.        ,
        304.13812651, 305.16389039],
       [239.        , 239.03765394, 238.26665734, ..., 304.13812651,
          0.        , 303.10889132],
       [229.77380181, 229.70415756, 229.77380181, ..., 305.16389039,
        303.10889132,   0.        ]])

In [381]:
df_matrix = pd.DataFrame(1/(1 + cs_matrix), 
                         index=df_pivot.columns, columns=df_pivot.columns)
df_matrix.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [399]:
similarities33 = df_matrix[33].sort_values(ascending=False)[1:]
top5_sim33=similarities33.head()
top5_sim33
top5_list =top5_sim33.index.to_list()
top5_list

[264, 3535, 3317, 2503, 3305]

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [400]:
df_33= df_q[df_q['CustomerID'].isin(top5_list)]
df_33

,CustomerID,ProductName,Quantity
131,264,Apricots - Halves,1.0
132,264,Apricots Fresh,1.0
133,264,Bacardi Breezer - Tropical,1.0
134,264,Bagel - Plain,1.0
135,264,Banana - Leaves,1.0
136,264,Bananas,1.0
137,264,Bandage - Flexible Neon,1.0
138,264,"Beef - Tenderlion, Center Cut",1.0
139,264,Berry Brulee,1.0
140,264,"Bread - Roll, Canadian Dinner",1.0


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [402]:
df_33_gb= df_33.groupby('ProductName').agg({'Quantity': 'sum'}).sort_values('Quantity', ascending= False)
df_33_gb.head()

,Quantity
ProductName,
Butter - Unsalted,3.0
Wine - Ej Gallo Sierra Valley,3.0
Towels - Paper / Kraft,3.0
Soup - Campbells Bean Medley,3.0
Wine - Blue Nun Qualitatswein,3.0


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [307]:
df_33_nobuy = df_pivot[df_pivot[33]==0]
merged= df_33_nobuy.merge(df_33_gb, left_on='ProductName', right_on='ProductName' ).sort_values('Quantity', ascending= False)
merged33= merged[33].head()
merged33

ProductName
Wine - Blue Nun Qualitatswein    0.0
Soup - Campbells Bean Medley     0.0
Wine - Ej Gallo Sierra Valley    0.0
Butter - Unsalted                0.0
Yoghurt Tubes                    0.0
Name: 33, dtype: float64

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [390]:
reco_dict={}
customers=list(data['CustomerID'].unique())

for cust in customers:
    similaritiesAll = df_matrix[cust].sort_values(ascending=False)[1:]
    top5_simAll=similaritiesAll.head()
    top5_list =top5_simAll.index.to_list()
    df_all= df_q[df_q['CustomerID'].isin(top5_list)]
    df_all_gb= df_all.groupby('ProductName').agg({'Quantity': 'sum'}).sort_values('Quantity', ascending= False)
    df_all_nobuy = df_pivot[df_pivot[cust]==0]
    merged= df_all_nobuy.merge(df_all_gb, left_on='ProductName', right_on='ProductName' ).sort_values('Quantity', ascending= False)
    mergedAll= merged[cust].head()
    reco_dict.update({cust: mergedAll})

reco_dict

{61288: ProductName
 Chicken - Soup Base          0.0
 Jagermeister                 0.0
 Lime Cordial - Roses         0.0
 Macaroons - Two Bite Choc    0.0
 Water - Mineral, Natural     0.0
 Name: 61288, dtype: float64, 77352: ProductName
 Wine - Valpolicella Masi    0.0
 Tahini Paste                0.0
 Tuna - Salad Premix         0.0
 Chicken - Soup Base         0.0
 Isomalt                     0.0
 Name: 77352, dtype: float64, 40094: ProductName
 Bread - Italian Corn Meal Poly    0.0
 Puree - Mocha                     0.0
 Tuna - Salad Premix               0.0
 Sherry - Dry                      0.0
 Beer - Sleemans Cream Ale         0.0
 Name: 40094, dtype: float64, 23548: ProductName
 Squid - Tubes / Tenticles 10/20    0.0
 Pernod                             0.0
 Wanton Wrap                        0.0
 Bread - Raisin Walnut Oval         0.0
 Banana Turning                     0.0
 Name: 23548, dtype: float64, 78981: ProductName
 Vanilla Beans                 0.0
 Lettuce - Frisee  

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [394]:
df_recomended= pd.DataFrame(reco_dict)
df_recomended.head()

,61288,77352,40094,23548,78981,83106,11253,35107,15088,26031,...,25270,17390,97029,38154,74403,49005,41286,85878,68506,3472
Anchovy Paste - 56 G Tube,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Appetizer - Mini Egg Roll, Shrimp",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Appetizer - Mushroom Tart,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Appetizer - Sausage Rolls,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Apricots - Dried,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [ ]:
for name, score in dict(similarities33).items():
    df_pivot[name] = df_pivot[name] * score

df_pivot['Total'] = df_pivot.sum(axis=1)
df_pivot.sort_values('Total', ascending=False)
df_pivot.head()

df_agg= df_pivot.groupby('ProductName').agg({'Total': 'sum'}).sort_values('Total', ascending= False)
df_agg.head()